# 実験を実行する

Azure Machine Learning SDK を使用して、メトリックを記録して出力を生成するコード実験を実行できます。これは、Azure Machine Learning におけるほとんどの機械学習操作の中核です。

## ワークスペースに接続する

すべての実験と関連リソースは、Azure Machine Learning ワークスペース内で管理されます。ほとんどの場合、ワークスペースの構成は JSON 構成ファイルに格納されます。これにより、Azure サブスクリプション ID などの詳細を覚えておく必要なく、簡単に再接続できます。Azure portal のワークスペースのブレードから JSON 構成ファイルをダウンロードできますが、ワークスペースでコンピューティング インスタンスを使用している場合、構成ファイルは既にルート フォルダーにダウンロードされています。

次のコードでは、構成ファイルを使用してワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験を実行する

データ サイエンティストが実行する必要のある最も基本的なタスクの 1 つは、データを処理して分析する実験を作成して実行することです。この演習では、Azure ML の*実験*を使用して、Python コードを実行し、データから抽出された値を記録する方法を学習します。この場合、糖尿病の検査を受けた患者の詳細を含む単純なデータセットを使用します。データを探索し、統計情報、視覚化、およびデータ サンプルを抽出する実験を実行します。使用するコードのほとんどは、データ探索プロセスで実行されるなど、かなり汎用的な Python です。ただし、数行を追加すると、コードは Azure ML *実験*を使用して実行の詳細を記録します。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# ワークスペースで Azure 実験を作成する
experiment = Experiment(workspace=ws, name="mslearn-diabetes")

# 実験データのログ記録を開始して、実験の実行レコードの参照を取得する
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# ローカル ファイルからデータを読み込む
data = pd.read_csv('data/diabetes.csv')

# 行をカウントし、結果を記録する
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# 糖尿病患者対非糖尿病患者の数をプロットし、記録する
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name='label distribution', plot=fig)

# 明確な妊娠数を記録する
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# 数値列の概要の統計情報を記録する
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# データのサンプルを保存し、実験出力にアップロードする
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

# 実行を完了する
run.complete()

## 実行の詳細を表示する

Jupyter Notebook では、**RunDetails** ウィジェットを使用して、実行の詳細を視覚化できます。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

### Azure Machine Learning Studio の詳細を表示する

**RunDetails** ウィジェットには、Azure Machine Learning Studio で**実行の詳細を表示**するためのリンクが含まれていることに注意してください。これをクリックすると、実行の詳細を表示する新しいブラウザー タブが開きます ([Azure Machine Learning Studio](https://ml.azure.com) を開いて、**実験**ページで実行を検索することもできます)。Azure Machine Learning Studio での実行を表示する場合は、次の点に注意してください。

- **「詳細」** タブには、実験実行の一般的なプロパティが含まれています。
- **「メトリック」** タブでは、記録されたメトリックを選択し、表またはグラフとして表示できます。
- **「イメージ」** タブでは、実験で記録された画像やプロットを選択して表示できます (この場合は、*ラベル分布*プロット)
- **「子の実行」** タブには、子の実行が一覧表示されます (この実験では、何も表示されません)。
- **「出力 + ログ」** タブには、実験で生成された出力ファイルまたはログ ファイルが表示されます。
- **「スナップショット」** タブには、実験コードが実行されたフォルダー内のすべてのファイル (この場合は、このノートブックと同じフォルダー内のすべてファイル) が含まれます。
- **「説明」** タブは、実験によって生成されたモデルの説明を表示するために使用されます (この場合は、何も表示されません)。
- **「均等化」** タブは、機械学習モデルの公平性の評価に役立つ予測パフォーマンスの格差を視覚化するために使用されます (この場合は、何も表示されません)。

### SDK を使用して実験の詳細を取得する

以前に実行したコードの **run** 変数は **Run** オブジェクトのインスタンスで、Azure Machine Learning で実験の個々の実行を参照します。この参照を使用すると、実行とその出力に関する情報を得られます。

In [ ]:
import json

# 指標の記録を取得する
print("Metrics:")
metrics = run.get_metrics()
for metric_name in metrics:
    print(metric_name, ":", metrics[metric_name])

# 出力ファイルを取得する
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)

実験で作成したファイルは、**download_file** メソッドを使用して個々に、または **download_files** メソッドを使用して複数のファイルを取得することでダウンロードできます。以下のコードを使って、実行の**output**フォルダーのファイルをすべてダウンロードします。

In [ ]:
import os

download_folder = 'downloaded-files'

# 「出力」フォルダーでファイルをダウンロードする
run.download_files(prefix='outputs', output_directory=download_folder)

# ファイルがダウンロードされていることを確認する
for root, directories, filenames in os.walk(download_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

実験実行のトラブルシューティングが必要な場合は、**get_details** メソッドを使用して実行に関する基本的な詳細を取得するか、**get_details_with_logs** メソッドを使用して実行の詳細と実行中に生成されたログ ファイルの内容を取得できます。

In [ ]:
run.get_details_with_logs()

詳細には、実験が実行されたコンピューティング ターゲット、実験の開始日時、終了日時などの情報が含まれることに注意してください。さらに、実験コードを含むノートブック (このノート) は複製された Git リポジトリにあるため、リポジトリ、ブランチ、状態に関する詳細が実行履歴に記録されます。

この場合は、詳細の **logFiles** エントリは、ログ ファイルが生成されていないことを示している点に留意してください。このようなインライン実験では、これは一般的なことですが、実験としてスクリプトを実行するとさらに興味深い状態になります。これからそれを見ていきましょう。

## 実験スクリプトを実行する

前の例では、このノートブックで実験をインラインで実行しました。より柔軟なソリューションとして、実験用の別のスクリプトを作成し、必要な他のファイルと共にフォルダーに格納し、Azure ML を使用してフォルダー内のスクリプトに基づいて実験を実行します。

まず、実験ファイルのフォルダーを作成し、データをコピーします。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

次に、実験用のコードを含む Python スクリプトを作成し、実験フォルダーに保存します。

> **注**: 次のセルを実行するとスクリプト ファイルが*作成されます*が、実行されません。

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
data = pd.read_csv('diabetes.csv')

# 行をカウントし、結果を記録する
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# ラベル数のカウントし記録する
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# 出力フォルダーにデータのサンプルを保存する (自動的にアップロードされます)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# 実行を完了する
run.complete()

このコードは、以前に使用したインライン コードの簡略化されたバージョンです。ただし、次の点に注意してください。
- スクリプトの実行時に実験の実行コンテキストを取得するために、`Run.get_context()` メソッドを使用します。
- スクリプトが配置されているフォルダーから糖尿病データを読み込みます。
- **outputs** という名前のフォルダーを作成し、サンプルファイルをそれに書き込みます - このフォルダーは自動的に実験実行にアップロードされます

これで、実験を実行する準備がほぼ整いました。スクリプトを実行するには、実験で実行する Python スクリプト ファイルを識別する **ScriptRunConfig** を作成し、それに基づいて実験を実行する必要があります。

> **注**: ScriptRunConfig は、コンピューティング ターゲットと Python 環境も決定します。これらを指定しない場合、デフォルト環境は、コードが実行されているローカル コンピューティング (この場合、このノートブックが実行されている場所) に自動的に作成されます。

次のセルは、スクリプトベースの実験を構成して送信します。

In [ ]:
import os
import sys
from azureml.core import Experiment, ScriptRunConfig
from azureml.widgets import RunDetails


# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder, 
                      script='diabetes_experiment.py') 

# 実験を送信する
experiment = Experiment(workspace=ws, name='mslearn-diabetes')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

以前と同様に、ウィジェットまたは [Azure Machine Learning Studio](https://ml.azure.com) の実験へのリンクを使用して、実験によって生成された出力を表示したり、生成されたメトリックとファイルを取得するコードを記述したりできます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

今回は実行によっていくつかのログ ファイルが生成されている点に留意してください。これはウィジェットで表示したり、**get_details_with_logs** メソッドを使用したりできますが、今回は出力にログ データが含まれます。

In [ ]:
run.get_details_with_logs()

ログの詳細は上記の出力で表示できますが、通常はログ ファイルをダウンロードしてテキスト エディタで表示する方が簡単です。

In [ ]:
import os

log_folder = 'downloaded-logs'

# すべてのファイルをダウンロードする
run.get_all_logs(destination=log_folder)

# ファイルがダウンロードされていることを確認する
for root, directories, filenames in os.walk(log_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

## 実験実行履歴を表示する

同じ実験を複数回実行したので、[Azure Machine Learning Studio](https://ml.azure.com) で履歴を表示し、記録された各実行を調べることができます。または、SDK を使用して、ワークスペースから名前で実験を取得し、実行を反復処理することもできます。

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['mslearn-diabetes']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

## MLflow を使用する

MLflow は、機械学習プロセスを管理するためのオープンソース プラットフォームです。これは、実験を調整し、メトリックを追跡するために、Databricks 環境で一般的に使用されます (ただし、排他的ではありません)。Azure Machine Learning の実験では、ネイティブ ログ機能の代わりに指標を追跡できます。

この機能を利用するには、**mlflow** と **azureml-mlflow** パッケージが必要なので、これらがインストールされていることを確認しましょう。

In [ ]:
!pip show mlflow azureml-mlflow

### インライン実験で MLflow を使用する

MLflow を使用してインライン実験の指標を追跡するには、実験が実行されているワークスペースに MLflow *トラッキング URI* を設定する必要があります。これにより、**mlflow** トラッキング メソッドを使用して、実験の実行にデータを記録できます。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import mlflow

# ワークスペースに MLflow トラッキング URI を設定する
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# ワークスペースで Azure 実験を作成する
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
mlflow.set_experiment(experiment.name)

# MLflow 実験を開始する
with mlflow.start_run():
    
    print("Starting experiment:", experiment.name)
    
    # データを読み込む
    data = pd.read_csv('data/diabetes.csv')

    # 行をカウントし、結果を記録する
    row_count = (len(data))
    mlflow.log_metric('observations', row_count)
    print("Run complete")

それでは実行中にログされた指標を見てみましょう。

In [ ]:
# 最新の実験を実行する
run = list(experiment.get_runs())[0]

# 指標の記録を取得する
print("\nMetrics:")
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
    
# Azure ML Studio で実験へのリンクを取得する   
experiment_url = experiment.get_portal_url()
print('See details at', experiment_url)

上記のコードを実行した後、表示されるリンクを使用して、Azure Machine Learning Studio で実験を表示できます。次に、実験の最新の実行を選択し、その 「**指標**」 タブを表示して、ログに記録された指標を確認します。

### 実験スクリプトで MLflow を使用する

MLflow を使用して、実験スクリプトの指標を追跡することもできます。

次の 2 つのセルを実行して、MLflow を使用する実験用のフォルダーとスクリプトを作成します。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
folder_name = 'mlflow-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

In [ ]:
%%writefile $folder_name/mlflow_diabetes.py
from azureml.core import Run
import pandas as pd
import mlflow


# MLflow 実験を開始する
with mlflow.start_run():
       
    # データを読み込む
    data = pd.read_csv('diabetes.csv')

    # 行をカウントし、結果を記録する
    row_count = (len(data))
    print('observations:', row_count)
    mlflow.log_metric('observations', row_count)

Azure ML 実験スクリプトで MLflow トラッキングを使用する場合、実験の実行を開始すると、MLflow トラッキング URI が自動的に設定されます。ただし、スクリプトを実行する環境には、必要な **mlflow** パッケージが含まれている必要があります。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 実験用 Python 環境を作成する
mlflow_env = Environment("mlflow-env")

# 必要なパッケージがインストールされていることを確認する
packages = CondaDependencies.create(conda_packages=['pandas','pip'],
                                    pip_packages=['mlflow','azureml-mlflow'])
mlflow_env.python.conda_dependencies = packages

# スクリプト構成を作成する
script_mlflow = ScriptRunConfig(source_directory=experiment_folder,
                                script='mlflow_diabetes.py',
                                environment=mlflow_env) 

# 実験を送信する
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
run = experiment.submit(config=script_mlflow)
RunDetails(run).show()
run.wait_for_completion()

いつものように、実験の実行が終了すると、ログに記録されたメトリックを取得できます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))

> **詳細情報**: 実験の実行の詳細については、Azure ML のドキュメントの[このトピック](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs)を参照してください。実行でのメトリックの記録方法の詳細については、[このトピック](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments)を参照してください。Azure ML の実験と MLflow の統合の詳細については、[このトピック](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-mlflow)を参照してください。